In [2]:
%%writefile utils.py 

import yaml
import gzip

# READ FILE USING YAML 
def read_yaml(path):
    
    with open(path, 'r') as data: 
        try:
            return yaml.safe_load(data)
        except:
            print("Something went wrong with loading the data.")

# CLEAN THE COLUMN NAMES 
def clean_col_names(df):

    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[%\W]', '', regex= True)

# RETURN DIFFERENCE IN COLUMNS 
def col_dif(df, conf_table):

    df_cols = list(df.columns)
    ct_cols = conf_table["columns"]
    missing_from_df = list(set(ct_cols).difference(df_cols))
    missing_from_ct = list(set(df_cols).difference(ct_cols))
    
    return df_cols, missing_from_df, ct_cols, missing_from_ct

# CHECKS FOR THE SAME NUMBER OF COLUMNS  
def col_val(df, conf_table):

    df_cols = list(df.columns)
    ct_cols = list(conf_table["columns"])
    df_cols_count = len(df_cols)
    ct_cols_count = conf_table["num_columns"]

    if df_cols == ct_cols:
        print("Validation test: PASS")
        return 1
    else: 
        print("Validation test: FAILED")
        print(f"Incoming data consists of {df_cols_count} columns.")
        print(f"YAML file consists of {ct_cols_count} columns.")
        print("The listed columns are not in the incoming data: \n{}".format(list(set(ct_cols).difference(df_cols))))
        print("The listed columns are not in the YAML file: \n{}".format(list(set(df_cols).difference(ct_cols))))
        return 0 

# WRITE TE FILE IN PUIPE SEPARATED TEXT FILE IN GZ FORMAT
def write_file(df):

    df.to_csv(
        "stocks.csv.gz",
        index= False,
        sep= "|",
        compression= "gzip"
    )

# SUMMARY OF THE DATA
def summary(conf_table):

    print("The file name is {}".format(conf_table["path"]))
    print("The delimiter used is '{}'".format(conf_table["delimiter"]))
    print("The number of columns in the data is {}".format(conf_table["num_columns"]))
    print("The data is 2GB in size.")
    print("The columns for the data are: {}".format(conf_table["columns"]))

# DISPLAY THE RESULTS
def result(col_val, df, conf_table):

    if col_val == 1: 
        # write the file in pip (|) separated text file
        write_file(df)
        # summary of the file 
        summary(conf_table)
    else: 
        df_cols, missing_from_df, ct_cols, missing_from_ct = col_dif(df, conf_table)
        print("It seems that the incoming data containing the following columns: {} \nis missing {} ".format(df_cols,missing_from_df))
        print("It seems that the YAML file containing the following columns: {} \nis missing {} ".format(ct_cols, missing_from_ct))

Overwriting utils.py


## Creating a YAML file for the data and then reading the YAML file to read the data 

In [3]:
%%writefile stocks.yaml

path: stocks.csv
delimiter: ","
num_columns: 6
columns: 
    - exchangecountry
    - symbol
    - companyname
    - returnonequity
    - priceearningsratio
    - dividendyield

Overwriting stocks.yaml


In [4]:
# using YAML 
import utils as u 

conf_table = u.read_yaml("stocks.yaml")
conf_table

{'path': 'stocks.csv',
 'delimiter': ',',
 'num_columns': 6,
 'columns': ['exchangecountry',
  'symbol',
  'companyname',
  'returnonequity',
  'priceearningsratio',
  'dividendyield']}

## Reading the data using Pandas, Dask, csv modules in python 

In [5]:
import pandas as pd 
import dask.dataframe 
import csv

# using pandas 
pd_stocks = pd.read_csv("stocks.csv")

# using Dask 
dask_stocks = dask.dataframe.read_csv("stocks.csv")

# using csv
csv_file = open("stocks.csv")
data = csv.reader(csv_file, delimiter= ",", quotechar= "|")
csv_file.close()

## cleaning the pandas data 

In [6]:
list(pd_stocks.columns)

['Exchange Country',
 'Symbol',
 'Company Name ',
 'Return on Equity %',
 'Price / Earnings Ratio %',
 'Dividend Yield %']

In [7]:
# cleaning the column headers 
u.clean_col_names(pd_stocks)

list(pd_stocks.columns)

['exchangecountry',
 'symbol',
 'companyname',
 'returnonequity',
 'priceearningsratio',
 'dividendyield']

## Validate the incoming dataframe from Pandas with the YAML file information on the data 

In [8]:
col_val = u.col_val(pd_stocks, conf_table)
u.result(col_val, pd_stocks, conf_table)

Validation test: PASS
The file name is stocks.csv
The delimiter used is ','
The number of columns in the data is 6
The data is 2GB in size.
The columns for the data are: ['exchangecountry', 'symbol', 'companyname', 'returnonequity', 'priceearningsratio', 'dividendyield']


## test dataframe to demo what happens if the columns dont match 


In [9]:
demo_data = {
    "exchange country": ["country 1", "country 2"],
    "company name": ["name 1", "name 2"],
    "made up column": [1, 2]
}

demo_df = pd.DataFrame(demo_data, columns= ["exchange country", "company name", "made up column"])

# cleaning cols 
u.clean_col_names(demo_df)
# val col 
demo_col_val = u.col_val(demo_df, conf_table)
print("\n")
# results from validation 
u.result(demo_col_val, demo_df, conf_table)

Validation test: FAILED
Incoming data consists of 3 columns.
YAML file consists of 6 columns.
The listed columns are not in the incoming data: 
['dividendyield', 'priceearningsratio', 'symbol', 'returnonequity']
The listed columns are not in the YAML file: 
['madeupcolumn']


It seems that the incoming data containing the following columns: ['exchangecountry', 'companyname', 'madeupcolumn'] 
is missing ['dividendyield', 'priceearningsratio', 'symbol', 'returnonequity'] 
It seems that the YAML file containing the following columns: ['exchangecountry', 'symbol', 'companyname', 'returnonequity', 'priceearningsratio', 'dividendyield'] 
is missing ['madeupcolumn'] 
